In [12]:
import numpy as np
import pandas as pd

from numpy.random import rand
from matplotlib import pyplot as plt

In [13]:
def initialstate(N):   
    ''' 
    generates a random spin configuration for the
    initial condition
    '''
    state = 2*np.random.randint(2, size=(N,N))-1
    return state

In [14]:
def mcmove(config, beta,field):
    '''Monte Carlo move using Metropolis algorithm '''
    for i in range(N):
        for j in range(N):
                a = np.random.randint(0, N)
                b = np.random.randint(0, N)
                s =  config[a, b]
                nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]
                cost = 2*s*nb - s*field
                if cost < 0:
                    s *= -1
                elif rand() < np.exp(-cost*beta):
                    s *= -1
                config[a, b] = s
    return config

In [15]:
def calcEnergy(config,field):
    '''Energy of a given configuration'''
    energy = 0
    for i in range(len(config)):
        for j in range(len(config)):
            S = config[i,j]
            nb = config[(i+1)%N, j] + config[i,(j+1)%N] + config[(i-1)%N, j] + config[i,(j-1)%N]
            energy += -nb*S - 4 * S * field
    return energy/4.

In [16]:
def calcMag(config):
    '''Magnetization of a given configuration'''
    mag = np.sum(config)
    return mag

In [22]:
def main(nt, address):
    counter = 0
    for tt in range(nt):
        counter += 1
        field = rand()*(-1)**np.random.randint(2)
        if tt%20==0: print('RUNNING ITERATION ',counter,' OF ',nt,' WITH FIELD INTENSITY ',field)
        E1 = M1 = E2 = M2 = 0
        config = initialstate(N)
        iT=1.0/T[tt]; iT2=iT*iT;

        for i in range(eqSteps):         
            mcmove(config, iT,field)           # Monte Carlo moves

        for i in range(mcSteps):
            mcmove(config, iT,field)           
            Ene = calcEnergy(config, field)     # calculate the energy
            Mag = calcMag(config)        # calculate the magnetisation

            E1 = E1 + Ene
            M1 = M1 + Mag
            M2 = M2 + Mag*Mag 
            E2 = E2 + Ene*Ene

        H[tt] = float(field)
        E[tt] = n1*E1
        M[tt] = n1*M1
        C[tt] = (n1*E2 - n2*E1*E1)*iT2
        X[tt] = (n1*M2 - n2*M1*M1)*iT
    if True:
        f = plt.figure(figsize=(18, 10)); # plot the calculated values    
        sp =  f.add_subplot(2, 2, 1 );
        plt.scatter(T, E, s=50, marker='o', color='IndianRed')
        plt.xlabel("Temperature (T)", fontsize=20);
        plt.ylabel("Energy ", fontsize=20);         plt.axis('tight');
        sp =  f.add_subplot(2, 2, 2 );
        plt.scatter(T, abs(M), s=50, marker='o', color='RoyalBlue')
        plt.xlabel("Temperature (T)", fontsize=20); 
        plt.ylabel("Magnetization ", fontsize=20);   plt.axis('tight');
        sp =  f.add_subplot(2, 2, 3 );
        plt.scatter(T, C, s=50, marker='o', color='IndianRed')
        plt.xlabel("Temperature (T)", fontsize=20);  
        plt.ylabel("Specific Heat ", fontsize=20);   plt.axis('tight');   
        sp =  f.add_subplot(2, 2, 4 );
        plt.scatter(T, X, s=50, marker='o', color='RoyalBlue')
        plt.xlabel("Temperature (T)", fontsize=20); 
        plt.ylabel("Susceptibility", fontsize=20);   plt.axis('tight');
        plt.savefig(address+'/ising.png')
    df = pd.DataFrame({'field':H ,'T':T,'E':E,'C':C,'X':X,'M':M}) # append
    with open(address+'/ising.csv', 'a') as f:
        df.to_csv(f, mode='a', header=f.tell()==0, index=False)

In [23]:
nt      = 6500
N       = 6        
eqSteps = 1024      
mcSteps = 1024       
T       = np.linspace(0.2, 3.5, nt); 
E,M,C,X,H = [np.zeros(nt) for _ in range(5)]
n1, n2  = 1.0/(mcSteps*N*N), 1.0/(mcSteps*mcSteps*N*N) 

from google.colab import drive
drive.mount('/content/drive')
import os
try:
  os.mkdir('drive/My Drive/ising')
except: pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
address = 'drive/My Drive/ising' #where the results will be saved
main(nt, address)

RUNNING ITERATION  1  OF  6500  WITH FIELD INTENSITY  -0.5368045268249428


In [20]:
# THIS WILL KEEP COLLAB RUNNING 
# paste it in the browser's console; alternatively keep visiting the script every <30mins
#(max time= 12hs)
jsfunc="""
function ClickConnect(){
  colab.config
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End");
};
setInterval(ClickConnect, 1000000)
"""